In [143]:
import math
from decimal import *
class ReliabilityCalculator:
    def __init__(self, system, n, tr, s, th, rr, ):
        self.mttfn = 1400000        
        self.n = n
        self.system = system
        self.tr = tr
        self.s = s # node total storage capacity
        self.th = th
        self.rr = rr
        
    def initDecStore(self, usage, vdSize = 20000 ):
        self.vdSize = vdSize
        self.processingTime = 0.01 # ms - time to process one vd (decide where to put)

        self.vdsCount = self.n *math.ceil(self.s * usage / (100*self.vdSize))
        freeVDsCount = self.n *( math.floor((self.s )/(self.vdSize) ))- self.vdsCount # total free vds slots in the system
        self.freeVDsCount = math.floor(freeVDsCount)
        
    def initFullBackup(self, b):
        self.b = b #number of times to visit getPANFBR function
        
    def getVDsInfo(self):
        return (self.vdsCount, self.freeVDsCount)
    
    def getTd(self):
        return self.vdsCount*self.processingTime/((self.n-1)*3600)
        
    def getMTTRn(self): #MTTR node
        if self.system == "decStore":
            mTTRn = self.s /(3600*self.th*self.rr*(self.n-1)) + self.getTd()
        else:
            mTTRn = (self.s /(3600*self.th*self.rr)) + self.tr
        return mTTRn
    
    def getMTTFn(self): #mTTF node
        return self.mttfn
    
    def getPANFBRs(self):
        pANFBER = 1
        if self.system == "raid5":
            pANFBER = self.getMTTRn()/(self.getMTTFn()/(self.n - 1))
        if self.system == "raid10":
            pANFBER = self.getMTTRn()/(self.getMTTFn())
        if self.system == "raid6":
            pANFBER = math.prod([(self.getMTTRn()*(self.n - i))/self.getMTTFn() for i in range (1,3)])#/2
        if self.system == "decStore":
            marc = self.getMARC()
            pANFBER = (((marc*self.getMTTRn())/self.getMTTFn())**marc) * (self.getMTTFn()/math.prod([self.n-i for i in range(1, marc+1)]))
        if self.system == "fullbackup":
            pANFBER = math.prod([(self.getMTTRn()*(self.b+1-i))/self.getMTTFn() for i in range(1, self.b)])
        return pANFBER
    
    def getMTTFs(self): #MTTF system
        MTTFs = Decimal(((self.getMTTFn())/self.n))* (1/Decimal(self.getPANFBRs()))
        return MTTFs
    def F(self, n):
        if n == 0:
            return self.freeVDsCount
        return self.F(n-1) - (self.vdsCount/(self.n-n+1)) - (self.F(n-1)/(self.n-n+1))
    
    def getMARC(self):
        for i in range (1, self.n-2):
            if self.F(i) < 0:
                return i
        return self.n-2
    
    def getReliability(self):
        return 100*(1-8760/self.getMTTFs())

    def getNinesCountForNumber(self, number):
        if number == 100:
            return 12
        if str(number).startswith("99"):
            counter = 0
            for i in str(number)[3:]:
                if i == "9":
                    counter +=1
                else:
                    return counter
        return 0
    
    def getAccuracyNines(self, reliabilityArr):
        return [self.getNinesCountForNumber(item) for item in reliabilityArr]

        
        

In [134]:
#raid5
system = ReliabilityCalculator("raid5", 6, 0, 2000000, 15, 0.5)
print(system.getMTTRn())
print(system.getMTTFs())

74.07407407407408
881999999.9999999771910615121


In [3]:
#raid5
system = ReliabilityCalculator("raid5", 6, 0, 2000000, 15, 0.5)
print(system.getMTTRn())
print(system.getMTTFs())

74.07407407407408
pANFBER: 74.07407407407408 2000000 5
1799999999.9999995


In [11]:
#raid10
system = ReliabilityCalculator("raid10", 6, 0, 2000000, 15, 0.2)
print(system.getMTTRn())
print(system.getMTTFs())
print(system.getMTTFn(), system.getMTTRn(), system.getMTTFn())

185.1851851851852
3599999999.999999
2000000 185.1851851851852 2000000


In [125]:
#raid6
system = ReliabilityCalculator("raid6", 6, 0, 2000000, 15, 0.2)
print(system.getMTTRn())
print(system.getMTTFs())

185.1851851851852
666791999999.9998882860414577


## tests start here

In [144]:
#decStore
reliability = []
#vary by n
n=3
for i in range (10):
    total = 2000000
    vdSize = 100000
    n = n+1
    usage = 70 #%
    system = ReliabilityCalculator("decStore", n, 0, total, 30, 0.5)
    system.initDecStore(usage=usage, vdSize=vdSize)
    reliability.append(system.getReliability())

print(reliability)
print(system.getMARC())
print(system.getVDsInfo())

print(system.getAccuracyNines(reliability))

[Decimal('99.99981834373629881522226954'), Decimal('99.99993613633567963164843821'), Decimal('99.99997057156168187558576198'), Decimal('99.99999999988173258898789605'), Decimal('99.99999999995136152080013525'), Decimal('99.99999999997708931606300204'), Decimal('99.99999999999999994463398222'), Decimal('99.99999999999999997602497409'), Decimal('99.99999999999999998863199949'), Decimal('99.99999999999999999420301052')]
4
(182, 78)
[3, 4, 4, 9, 10, 10, 16, 16, 16, 17]


In [145]:
#decStore
reliability = []
#vary by n
n=3
for i in range (10):
    total = 2000000
    vdSize = 100000
    n = n+1
    usage = 80 #%
    system = ReliabilityCalculator("decStore", n, 0, total, 30, 0.5)
    system.initDecStore(usage=usage, vdSize=vdSize)
    reliability.append(system.getReliability())

print(reliability)
print(system.getMARC())
print(system.getVDsInfo())

print(system.getAccuracyNines(reliability))

[Decimal('89.70012692768959423363282570'), Decimal('99.99993613623988460219777674'), Decimal('99.99997057150871099649252679'), Decimal('99.99998410494340907289971861'), Decimal('99.99999046691527537604038190'), Decimal('99.99999384164972663296375409'), Decimal('99.99999579492829773036347856'), Decimal('99.99999999999330932618651239'), Decimal('99.99999999999601177888765711'), Decimal('99.99999999999750403507186511')]
3
(208, 52)
[0, 4, 4, 4, 5, 5, 5, 11, 11, 11]


In [146]:
#vary by n
reliability = []
n=3
for i in range (10):
    #decStore
    total = 2000000
    n = n+1
    usage = 90 #%
    vdSize = 100000
    system = ReliabilityCalculator("decStore", n, 0, total, 30, 0.5)
    system.initDecStore(usage=usage, vdSize=vdSize)
    reliability.append(system.getReliability())

print(reliability)
print(system.getMARC())
print(system.getVDsInfo())
print(system.getAccuracyNines(reliability))


[Decimal('89.70012074779541429896718390'), Decimal('92.75788762400793606869139499'), Decimal('94.43805018666666674479816009'), Decimal('95.49378457716049412251635431'), Decimal('96.21635045545837339377369192'), Decimal('96.74103183258928569313725849'), Decimal('99.99999579491568265717586309'), Decimal('99.99999700260780576475311123'), Decimal('99.99999778895409463437426386'), Decimal('99.99999832273119235753905147')]
2
(234, 26)
[0, 0, 0, 0, 0, 0, 5, 5, 5, 5]


In [148]:
#3-levels backup
reliability = []
total = 2000000
n = 3
for i in range (10):
    n = n+1
    system = ReliabilityCalculator("fullbackup", n, 0, total, 30, 0.5)
    system.initFullBackup(3)
    reliability.append(system.getReliability())

print(reliability)
print(system.getAccuracyNines(reliability))


[Decimal('99.99999998948997588453370648'), Decimal('99.99999998686246985566713309'), Decimal('99.99999998423496382680056037'), Decimal('99.99999998160745779793398633'), Decimal('99.99999997897995176906741295'), Decimal('99.99999997635244574020084055'), Decimal('99.99999997372493971133426619'), Decimal('99.99999997109743368246769431'), Decimal('99.99999996846992765360112074'), Decimal('99.99999996584242162473454463')]
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7]


In [149]:
#2-levels backup
reliability = []
total = 2000000
n = 3
for i in range (10):
    n = n+1
    system = ReliabilityCalculator("fullbackup", n, 0, total, 30, 0.5)
    system.initFullBackup(2)
    reliability.append(system.getReliability())

print(reliability)
print(system.getAccuracyNines(reliability))


[Decimal('99.99986757369614512471101481'), Decimal('99.99983446712018140588876851'), Decimal('99.99980136054421768707478103'), Decimal('99.99976825396825396824427592'), Decimal('99.99973514739229024942202962'), Decimal('99.99970204081632653061217154'), Decimal('99.99966893424036281177753702'), Decimal('99.99963582766439909297421717'), Decimal('99.99960272108843537414956205'), Decimal('99.99956961451247165529290404')]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [151]:
#1-levels backup
reliability = []
total = 2000000
n = 3
for i in range (10):
    n = n+1
    system = ReliabilityCalculator("fullbackup", n, 0, total, 30, 0.5)
    system.initFullBackup(1)
    reliability.append(system.getReliability())

print(reliability)
print(system.getAccuracyNines(reliability))


[Decimal('97.49714285714285714285714286'), Decimal('96.87142857142857142857142857'), Decimal('96.24571428571428587037727845'), Decimal('95.6200'), Decimal('94.99428571428571428571428571'), Decimal('94.36857142857142880556591767'), Decimal('93.74285714285714285714285714'), Decimal('93.11714285714285750056697740'), Decimal('92.49142857142857174075455690'), Decimal('91.86571428571428537608732526')]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [152]:
#raid5
reliability = []
total = 2000000
n = 3
for i in range (10):
    n = n+1
    system = ReliabilityCalculator("raid5", n, 0, total, 30, 0.5)
    reliability.append(system.getReliability())

print(reliability)
print(system.getAccuracyNines(reliability))

[Decimal('99.99980136054421768707500222'), Decimal('99.99966893424036281177753702'), Decimal('99.99950340136054421767423255'), Decimal('99.99930476190476190476250778'), Decimal('99.99907301587301587292622361'), Decimal('99.99880816326530612244868616'), Decimal('99.99851020408163265306251668'), Decimal('99.99817913832199546482444581'), Decimal('99.99781496598639455805155157'), Decimal('99.99741768707482993186766435')]
[3, 3, 3, 3, 3, 2, 2, 2, 2, 2]


In [153]:
#raid6
reliability = []
total = 2000000
n = 3
for i in range (10):
    n = n+1
    system = ReliabilityCalculator("raid6", n, 0, total, 30, 0.5)
    reliability.append(system.getReliability())

print(reliability)
print(system.getAccuracyNines(reliability))

[Decimal('99.99999998948997588453370648'), Decimal('99.99999997372493971133426619'), Decimal('99.99999994744987942266853249'), Decimal('99.99999990803728898966994253'), Decimal('99.99999985285966238347191136'), Decimal('99.99999977928949357520788864'), Decimal('99.99999968469927653601117356'), Decimal('99.99999956646150523701542034'), Decimal('99.99999942194867364935393194'), Decimal('99.99999924853327574416015669')]
[7, 7, 7, 7, 6, 6, 6, 6, 6, 6]
